In [5]:
import json
from tqdm import tqdm
import torch
from sentence_transformers import util

In [6]:
big = "starwars"
small = "swg"

mapping_file_big = "./data/triples_v2/" + big + "_mapping.json"
mapping_file_small = "./data/triples_v2/" + small + "_mapping.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communitiy_embeddings_big_file = "./results/community_embeddings/" + big + ".json"

node_embeddings_small_file = "./results/embeddings/" + small + "_lab_altlab_type_abs_comment_BAAI_bge-large-en-v1.5.json"
url_embeddings_small_file = "./results/url_embeddings_short/" + small + "_url_BAAI_bge-large-en-v1.5.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"
gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"

top10pairs_file = "./results/top10pairs/" + small + "-" + big + "_top10pairs.json"

In [7]:
big_communities = []
gold_pairs = []

with open(communities_big_file) as cbf, open(gold_pairs_file) as gpf:

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)

    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

with open(communitiy_embeddings_big_file) as cebf:
    community_embeddings_big = json.load(cebf)
    community_embeddings_big = {k: list(v.values()) for k, v in community_embeddings_big.items()}

with open(top10pairs_file) as file:
    top10pairs = json.load(file)

with open(node_embeddings_small_file) as nesf:
    node_embeddings_small = json.load(nesf)

with open(url_embeddings_small_file) as uesf:
    url_embeddings_small = json.load(uesf)

merged_node_embeddings_small = {key: node_embeddings_small[key] if key in node_embeddings_small else url_embeddings_small[key] for key in set(node_embeddings_small) | set(url_embeddings_small)}
merged_node_embeddings_small = {str(k): merged_node_embeddings_small[str(k)] for k in sorted(map(int, merged_node_embeddings_small.keys()))}

In [8]:
top1dict = dict()

for k, v in top10pairs.items():
    top1dict[int(k)] = int(v[0][0])

In [14]:
found = 0
all_pairs = len(gold_pairs)
# all_pairs = len(gold_not_exact)

for gold_pair in tqdm(gold_pairs):
# for gold_pair in tqdm(gold_not_exact):
    if top1dict.get(gold_pair[1]) == gold_pair[0]:
        found += 1
    # else:
    #     print(gold_pair)

print("Gold pairs in top 1: " + str(found / all_pairs * 100) + "% (" + str(found) + ")")

100%|██████████| 1121/1121 [00:00<00:00, 777930.97it/s]

Gold pairs in top 1: 72.97056199821587% (818)


In [15]:
node_to_community_embeddings = dict()
index = 0
for community in big_communities:
    for node in community:
        node_to_community_embeddings[node] = community_embeddings_big[str(index)]
    index += 1

In [20]:
top1dict_reordered = dict()

for k in tqdm(top10pairs.keys()):

    node_embeds = torch.Tensor(merged_node_embeddings_small[str(k)])
    # compare_list = [node_to_community_embeddings[int(item[0])] for item in top10pairs[str(k)]]
    compare_list = [
        node_to_community_embeddings.get(int(item[0]))
        for item in top10pairs[str(k)]
        if int(item[0]) in node_to_community_embeddings
    ]
    big_torch_embeds = torch.Tensor(compare_list)

    community_order = util.semantic_search(node_embeds, big_torch_embeds)

    best_score = 0

    for item in community_order[0]:
        score = top10pairs[str(k)][item['corpus_id']][1] + 0.2 * item['score']
        id_node = top10pairs[str(k)][item['corpus_id']][0]
        if score > best_score:
            best_score = score
            top1dict_reordered[int(str(k))] = int(id_node)

100%|██████████| 10497/10497 [00:04<00:00, 2241.95it/s]


In [21]:
found = 0
all_pairs = len(gold_pairs)
# all_pairs = len(gold_not_exact)

for gold_pair in tqdm(gold_pairs):
# for gold_pair in tqdm(gold_not_exact):
    if top1dict_reordered.get(gold_pair[1]) == gold_pair[0]:
        found += 1

print("Gold pairs in reordered top 1: " + str(found / all_pairs * 100) + "% (" + str(found) + ")")

100%|██████████| 1121/1121 [00:00<00:00, 988606.98it/s]

Gold pairs in reordered top 1: 71.72167707404104% (804)


In [9]:
# node_embeds = torch.Tensor(merged_node_embeddings_small['2440'])
# compare_list = [node_to_community_embeddings[int(item[0])] for item in top10pairs['2440']]
# big_torch_embeds = torch.Tensor(compare_list)
#
# community_order = util.semantic_search(node_embeds, big_torch_embeds)

In [10]:
# top10pairs['2440']

In [11]:
# community_order

In [12]:
# top1dict_reordered = dict()
# best_score = 0
#
# for item in community_order[0]:
#     score = top10pairs['2440'][item['corpus_id']][1] + 0.5 * item['score']
#     id_node = top10pairs['2440'][item['corpus_id']][0]
#     print(id_node, score)
#     if score > best_score:
#         best_score = score
#         top1dict_reordered[int('2440')] = int(id_node)

In [13]:
# top1dict_reordered